In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
scaler = StandardScaler()

In [28]:
# Novos dados
novos_dados = {'Renda_Anual':	100000,
'Tipo_Moradia':	'Própria',
'Anos_no_Emprego': 7.0,
'Motivo_Emprestimo':	'Empreendimento',
'Grau_Emprestimo': 'A',
'Valor':	3500,
'Taxa_Juros': 8.78,
'Comprometimento_Renda':	0.04,
'Inadimplencia_Historica':	'Sim',
}

In [125]:
data_exemplo = pd.DataFrame({
    'Renda_Anual': [50000, 60000, 70000],
    'Anos_no_Emprego': [5, 10, 15],
    'Grau_Emprestimo': [1, 2, 3],
    'Valor': [1000, 2000, 3000],
    'Taxa_Juros': [5.0, 6.0, 7.0],
    'Comprometimento_Renda': [0.1, 0.2, 0.3],
    'Inadimplencia_Historica': [0, 1, 0]  # 0 = Não, 1 = Sim
})

# Ajustar o scaler
scaler = StandardScaler()
scaler.fit(data_exemplo)

def preprocess_new_data(new_data_dict):
    # Converter o dicionário para DataFrame
    new_data = pd.DataFrame([new_data_dict])
    
    # Converter os tipos de dados conforme necessário
    new_data['Renda_Anual'] = new_data['Renda_Anual'].astype(float)
    new_data['Anos_no_Emprego'] = new_data['Anos_no_Emprego'].astype(int)
    
    # Mapear Grau_Emprestimo de letras para números
    new_data['Grau_Emprestimo'] = new_data['Grau_Emprestimo'].map({'A': 1, 'B': 2, 'C': 3})
    new_data['Valor'] = new_data['Valor'].astype(float)
    new_data['Taxa_Juros'] = new_data['Taxa_Juros'].astype(float)
    new_data['Comprometimento_Renda'] = new_data['Comprometimento_Renda'].astype(float)
    
    # Mapeamento de Inadimplencia_Historica de 'Sim'/'Não' para 1/0
    new_data['Inadimplencia_Historica'] = new_data['Inadimplencia_Historica'].map({'Sim': 1, 'Não': 0})

    # Criar colunas dummy manualmente para Tipo_Moradia
    new_data['Tipo_Moradia_Hipoteca'] = 0
    new_data['Tipo_Moradia_Outro'] = 0
    new_data['Tipo_Moradia_Próprio'] = 1 if new_data_dict['Tipo_Moradia'] == 'Própria' else 0
    
    # Criar colunas dummy manualmente para Motivo_Emprestimo
    new_data['Motivo_Emprestimo_Educação'] = 1 if new_data_dict['Motivo_Emprestimo'] == 'Educação' else 0
    new_data['Motivo_Emprestimo_Empreendimento'] = 1 if new_data_dict['Motivo_Emprestimo'] == 'Empreendimento' else 0
    new_data['Motivo_Emprestimo_Médico'] = 1 if new_data_dict['Motivo_Emprestimo'] == 'Médico' else 0
    new_data['Motivo_Emprestimo_Pessoal'] = 1 if new_data_dict['Motivo_Emprestimo'] == 'Pessoal' else 0
    new_data['Motivo_Emprestimo_Reformas Domésticas'] = 1 if new_data_dict['Motivo_Emprestimo'] == 'Reformas Domésticas' else 0

    # Selecionar as colunas numéricas para escalar
    cols_to_scale = ['Renda_Anual', 'Anos_no_Emprego', 'Grau_Emprestimo', 
                     'Valor', 'Taxa_Juros', 'Comprometimento_Renda', 
                     'Inadimplencia_Historica']
    
    # Aplicar o scaler às colunas selecionadas
    new_data[cols_to_scale] = scaler.transform(new_data[cols_to_scale])
    
    # Reorganizar as colunas para corresponder ao seu DF original
    column_order = ['Renda_Anual', 'Anos_no_Emprego', 'Grau_Emprestimo', 'Valor', 
                    'Taxa_Juros', 'Comprometimento_Renda', 'Inadimplencia_Historica'] + \
                   [col for col in new_data.columns if col not in ['Renda_Anual', 'Anos_no_Emprego', 
                                                                     'Grau_Emprestimo', 'Valor', 
                                                                     'Taxa_Juros', 'Comprometimento_Renda', 
                                                                     'Inadimplencia_Historica']]
    new_data = new_data[column_order]
    
    new_data = new_data.drop('Tipo_Moradia', axis=1)
    new_data = new_data.drop('Motivo_Emprestimo', axis=1)
    
    return new_data


# Processar os novos dados
preprocessed_data = preprocess_new_data(novos_dados)
print(preprocessed_data)

   Renda_Anual  Anos_no_Emprego  Grau_Emprestimo     Valor  Taxa_Juros  \
0     4.898979        -0.734847        -1.224745  1.837117    3.404791   

   Comprometimento_Renda  Inadimplencia_Historica  Tipo_Moradia_Hipoteca  \
0              -1.959592                 1.414214                      0   

   Tipo_Moradia_Outro  Tipo_Moradia_Próprio  Motivo_Emprestimo_Educação  \
0                   0                     1                           0   

   Motivo_Emprestimo_Empreendimento  Motivo_Emprestimo_Médico  \
0                                 1                         0   

   Motivo_Emprestimo_Pessoal  Motivo_Emprestimo_Reformas Domésticas  
0                          0                                      0  


In [126]:
preprocessed_data

,Renda_Anual,Anos_no_Emprego,Grau_Emprestimo,Valor,Taxa_Juros,Comprometimento_Renda,Inadimplencia_Historica,Tipo_Moradia_Hipoteca,Tipo_Moradia_Outro,Tipo_Moradia_Próprio,Motivo_Emprestimo_Educação,Motivo_Emprestimo_Empreendimento,Motivo_Emprestimo_Médico,Motivo_Emprestimo_Pessoal,Motivo_Emprestimo_Reformas Domésticas
0,4.898979,-0.734847,-1.224745,1.837117,3.404791,-1.959592,1.414214,0,0,1,0,1,0,0,0


In [127]:
melhor_modelo = joblib.load("C:/Projetos Pessoais/DataScience/analise_Risco_Credito/models/modelo_v6_adaboost.pkl")
melhor_modelo

AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(max_depth=10,
                                                         min_samples_split=3),
                   learning_rate=0.5, n_estimators=300)

In [128]:
# Realizando a previsão 
nova_previsao = melhor_modelo.predict(preprocessed_data)
nova_previsao_proba = melhor_modelo.predict_proba(preprocessed_data)

print(nova_previsao)
print(nova_previsao_proba)

[0]
[[0.55043238 0.44956762]]


In [130]:
cliente = pd.DataFrame([novos_dados])

In [139]:
cliente['Chances Inadimplência'] = round(nova_previsao_proba[0 , 0] * 100,2)

In [140]:
nova_previsao_proba[0 , 0]

0.5504323814655815

In [141]:
cliente

,Renda_Anual,Tipo_Moradia,Anos_no_Emprego,Motivo_Emprestimo,Grau_Emprestimo,Valor,Taxa_Juros,Comprometimento_Renda,Inadimplencia_Historica,Chances Inadimplência
0,100000,Própria,7.0,Empreendimento,A,3500,8.78,0.04,Sim,55.04
